In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import datasets

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
#display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
#display(HTML("<style>.prompt { display:none !important; }</style>"))



In [23]:
class Dense():

    def __init__(self, sizes, activations, Loss, epochs, metric, learning_rate):
        self.weights =[np.random.randn(sizes[i],sizes[i-1]) for i in range(1, len(sizes))]
        self.biases =  [np.zeros((1, sizes[i])) for i in range(1, len(sizes))]
        self.activations = activations
        self.Loss = Loss
        self.epochs = epochs
        self.metric = metric
        self.learning_rate = learning_rate
        
    def forward(self, inputs):
        x = inputs
        self.activated_layers = [x]
        for w,b, act in zip(self.weights, self.biases, self.activations):
            activation = act.activate(np.dot(x, w.T) + b)
            self.activated_layers.append(activation)
            x = activation
            
            
    def backpropagate(self, y):
        #Input: activated_layers
        #output: Container Gradient dE/dw 
        #Initialize
        sigmas_box = []
        sigma_prime_box = []
    
       #Backprop the error ()
        #1. Compute output layer sigma
        loss_grad = self.Loss.loss_gradient(self.activated_layers[-1], y)
        output_sigma = self.activations[-1].output_layer_sigma(loss_grad, self.activated_layers[-1])
        sigmas_box = [output_sigma]
        #Sigmas of the rest of layers...
        for w,a,o_layer in zip(self.weights[::-1], self.activations[:-1][::-1],self.activated_layers[:-1][::-1]) :
            sigmas_box.append(np.dot(sigmas_box[-1], w) * a.sigma_prime(o_layer))
       
        #Reverse sigma_box    
        sigmas_box.reverse()
        #Biases update
        self.grad_biases = sigmas_box
        #Gradient (dE/dw):
        self.gradients = []
        for a, s in zip(self.activated_layers[:-1], sigmas_box):
            self.gradients.append(np.dot(s.T, a))
        #Nota: al hacerse la multiplicacion de todos los inputs a la vez
        #Igual se mantiene el shape de cada weight pero mientras más
        #Inputs más grandes salen los valores de cada componente de la matriz
        #Por eso después se divide cada weight por el total de inputs (mean)
        # print("Gradients shapes:")
        # for g in gradients:
        #     print(g.shape)
       

    def weight_update(self):
        for w,gw, b, gb in zip(self.weights, self.gradients, self.biases, self.grad_biases):
            w -= (self.learning_rate / len(X))* gw
            b -= (self.learning_rate / len(gb))* np.sum(gb, axis= 0)

        return self.weights, self.biases
        
    
    def predict(self, X,y):
            self.forward(X)
            self.prediction = self.activated_layers[-1]
            #Compute the accuracy
            acc = self.metric.get_accuracy(self.prediction, y)
            print("Predictive accuracy:", acc)


            
    def SGD(self, X,y,x_test, y_test, minibatch_size):
        """Vectorized version"""
        print("SGD Training......")
        for e in range(1, self.epochs + 1):
            #tomar dataset y generar minibatches box
            minibatches = self.minibatch_generator(X,y, minibatch_size)
            Losses = []
            Accuracies = []
            for mb in minibatches:
                nabla_w, nabla_b = [], [] #box para ir guardando los dC/dw y dC/db de cada ejemplo
                input = mb[0]
                y_true = np.array(mb[1]).astype(int)
                #Dar un forward pass 
                #print("Minibatch input shape:", input.shape)
                #print("Minibatch y_true shape:", y_true.shape)
                #print("Bias", self.biases[0])

                
                self.forward(input)
                #Calcular el error
                error = self.Loss.forward_loss(self.activated_layers[-1], y_true)
                #Guardar el error y accuracy
                Losses.append(error)
                Accuracies.append(self.metric.get_accuracy(self.activated_layers[-1], y_true))


                #Obtener los dC/dw y dC/db del minibatch usando backprop
                self.backpropagate(y_true)
                delta_nw = self.gradients #dC/dw
                delta_nb = self.grad_biases #dC/db
                self.weights = [w - (self.learning_rate/ len(mb)) * dw for w,dw in zip(self.weights, delta_nw)]
                self.biases = [b - (self.learning_rate/ len(mb)) * np.sum(db, axis = 0) for b, db in zip(self.biases, delta_nb)]
            
            #Reporte de error epoch...
            if (e % 100 == 0 ) or (e == self.epochs):
                print("Average Error epoch {0}: {1}---Average Accuracy: {2}".format(e, np.mean(Losses), np.mean(Accuracies)))
                print("Accuracy in test set:", self.evaluate_test(x_test, y_test))
                   
        print("Training complete!")

    
            
    def Momentum(self, X,y,x_test, y_test, minibatch_size, beta = 0.9):
        """Vectorized version"""
        print("SGD Training......")
        for e in range(1, self.epochs + 1):
            #tomar dataset y generar minibatches box
            minibatches = self.minibatch_generator(X,y, minibatch_size)
            Losses = []
            Accuracies = []
            #Velocities initialization
            Vdw = [np.zeros(w.shape) for w in self.weights]
            Vdb = [np.zeros(b.shape) for b in self.biases]

            for mb in minibatches:
                nabla_w, nabla_b = [], [] #box para ir guardando los dC/dw y dC/db de cada ejemplo
                input = mb[0]
                y_true = np.array(mb[1]).astype(int)
                #Dar un forward pass 
                #print("Minibatch input shape:", input.shape)
                #print("Minibatch y_true shape:", y_true.shape)
                #print("Bias", self.biases[0])
                
                self.forward(input)
                #Calcular el error
                #error = np.mean(self.Loss.forward_loss(self.activated_layers[-1], y_true))
                error = self.Loss.forward_loss(self.activated_layers[-1], y_true)

                #Guardar el error y accuracy
                Losses.append(error)
                Accuracies.append(self.metric.get_accuracy(self.activated_layers[-1], y_true))

                #Obtener los dC/dw y dC/db del minibatch usando backprop
                self.backpropagate(y_true)
                delta_nw = self.gradients #dC/dw
                delta_nb = self.grad_biases #dC/db
                
                #Compute the exp moving averages (velocities)
                Vdw = [beta * vw + (1 - beta) * dnw for vw, dnw in zip(Vdw, delta_nw)]
                Vdb = [beta * vb + (1 - beta) * dnb for vb, dnb in zip(Vdb, delta_nb)]
                
                #Update weights and biases using the Velocities
                
                self.weights = [w - (self.learning_rate/ len(mb)) * dw for w,dw in zip(self.weights, Vdw)]
                self.biases = [b - (self.learning_rate/ len(mb)) * np.sum(db, axis = 0) for b, db in zip(self.biases, Vdb)]
            
            #Reporte de error epoch...
            if (e % 100 == 0 ) or (e == self.epochs):
                print("Average Error epoch {0}: {1}---Average Accuracy: {2}".format(e, np.mean(Losses), np.mean(Accuracies)))
                print("Accuracy in test set:", self.evaluate_test(x_test, y_test))
                   
        print("Training complete!") 

    def minibatch_generator(self, X,y, batch_size):
        dataset = list(zip(X,np.array(y)))
        np.random.shuffle(dataset)
        minibatches = [(X[i:i+batch_size,:], y[i:i+batch_size]) for
                        i in range(0, len(y), batch_size)]

        #si minibatch final es mas chico que el batch size se le mete desde
        #atras inputs hasta completar el tamaño batch size
        if len(minibatches[-1][0]) < batch_size:
            #print("Len minibatches -1:", len(minibatches[-1][0]))
            minibatches[-1] = (X[-batch_size:,:], y[-batch_size:])
            
        return minibatches
    
    def evaluate_test(self, x_test, y_test):
        """Evaluates the model on the test set
        input: x_test, y_test
        output: accuracy"""
        #Forward pass---obtain prediction y_pred
        self.forward(x_test)
        #Evaluate prediction with accuracy
        acc_test = self.metric.get_accuracy(self.activated_layers[-1], y_test)
        #Return accuracy
        return acc_test

        
 
class Relu():
    def activate(self, x):
        self.output = np.maximum(0,x)
        return self.output
    
    def sigma_prime(self, x):
        return 1. * (x > 0)


class Sigmoid():
    def activate(self, x):
        #np.exp - (x - np.max(x, axis = 1, keepdims= True))
        x = np.clip(x, 1e-7, 1 - 1e-7)
        self.output = 1 / (1+ np.exp (- (x - np.max(x, axis = 1, keepdims= True))))
        #self.output = 1 / (1+ np.exp(-x))
        return self.output
    
    def output_layer_sigma(self, loss_gradients, x):
        """en realidad calcula todo el sigma de una vez como dC/da * sigma_prime
        dC/da = loss_gradient"""
        
        self.output_sigma = loss_gradients * self.sigma_prime(x)
        return self.output_sigma
    
    def sigma_prime(self, x):
        return x * (1-x)

class Softmax():
    def activate(self, x):
        #Get unnormalized probs
        exp_values = np.exp(x - np.max(x, axis = 1, keepdims= True))
        #Get normalized probs
        self.output = exp_values / np.sum(exp_values, axis= 1, keepdims= True)
        return self.output
    
    def output_layer_sigma(self, loss_gradients, out_activations):
        """Dado que es complejo multplicar el jacobiano de cada input por
        #su loss_gradient por que el jac es una matrix, se hace aca todo directo y se saca 
        #el output layer sigma = dE/dsigma.dsigma/dz"""
        #Se crea un contenedor donde irá el output_sigma de cada input
        #del tamaño del loss_gradient (dinputs)
        self.output_sigma = np.empty_like(loss_gradients)

        #Tomo uno a uno los Loss_gradientes de cada input y cada
        #softmax activation de la output layer para hacer uno a uno los
        #output_sigmas...
        for index, (single_act, single_loss_grad) in enumerate(zip(out_activations, loss_gradients)):
            single_act = single_act.reshape(-1,1)
            #Calculate jacobian matrix (sigma_prime of softmax)
            jacobian_matrix = np.diagflat(single_act) - np.dot(single_act, single_act.T)
            self.output_sigma[index] = np.dot(jacobian_matrix, single_loss_grad)
        return self.output_sigma

         

    
##Loss Units
class MSE():
    
    #Forward
    def forward_loss(self, y_pred, y_true):
        y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
        labels = len(y_pred[0])                  
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        #return  np.sum((y_pred- y_true)**2, axis=1) / len(y_pred)
        return ((y_pred- y_true)**2).mean()
        
    #Derivative
    def loss_gradient(self, y_pred, y_true): #dE/dact
        y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
        labels = len(y_pred[0])                  
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        self.dinputs = (2/len(y_pred)) * (y_pred - y_true)
        return self.dinputs
    
    
class CategoricalCrossEntropyLoss():
    def forward_loss(self, y_pred, y_true):
         #entrega el vector de negative losses de cada sample
         y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7) #recorta para evitar logs mulas
         if len(y_true.shape) == 1: #si el y_true viene en un solo vector de escalares
             #extraigo el valor que tiene el indice indicado en el y_true
             #correspondiente
             correct_confidences = y_pred[range(len(y_pred)), y_true]
        
         if len(y_true.shape) == 2: #matrix
             #lo mismo pero multiplique y sume para obtener el valor
             #que tiene el indice indicado por el y_true (el resto se hace zero
             #al multiplicar)
             correct_confidences = np.sum( y_pred * y_true, axis = 1)
        
         negative_loss_likehoods = -np.log(correct_confidences)

         return negative_loss_likehoods
    
    def loss_gradient(self, dvalues, y_true): #dE/dact
        # Number of samples
        dvalues = np.clip(dvalues, 1e-7, 1 - 1e-7)
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])
        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        # Calculate gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples
        return self.dinputs



class Accuracy():
#gets the accuracy of the training stage
    def get_accuracy(self, y_pred, y_true):
        #saca el indice donde esta el valor mas grande
        predictions = np.argmax(y_pred, axis= 1)

        #y_true en formato escalares
        if len(y_true.shape) == 1:
            accuracy = np.mean(predictions == y_true)
        #matrix
        elif len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis= 1)
            accuracy = np.mean(predictions == y_true) #promedia coincidencias de valor de indice

        return accuracy



In [5]:
# train = pd.read_csv("mnist_train.csv", delimiter= ",")
# X = train.iloc[:, 1:]
# y = train.iloc[:, 0]

# #Loading test set
# test = pd.read_csv("mnist_test.csv", delimiter= ",")
# x_test = test.iloc[:, 1:]
# y_test = test.iloc[:, 0]

# # scaler = preprocessing.StandardScaler().fit(X)

# # X = scaler.transform(X)
# # test_X = scaler.transform( test_X )

# scaler = preprocessing.StandardScaler()
# X = scaler.fit_transform( X )
# x_test = scaler.transform(x_test)

In [35]:
# sizes2 = [784,30, 10]
# EPOCHS = 3000
# #sizes2 = [4, 5,  3]
# net = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
#             epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.9)
# #net = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = MSE(),
#             #epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.07)
# #net.train(X,y)

# #net.SGD(X , np.array(y), x_test, y_test, minibatch_size = 10)
# #net.SGD(X[:10000] , y[:10000], x_test, y_test, minibatch_size = 10)
# #net.Momentum(X[:10000] , y[:10000], x_test, y_test, minibatch_size = 10, beta = 0.9)
# #net.SGD(X[:1000] , y[:1000], x_test, y_test, minibatch_size = 10)

In [36]:
# net1 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = MSE(),
#             epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.05)
        
# net2 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
#             epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.05)

In [144]:
# train = pd.read_csv("mnist_train.csv", delimiter= ",")
# X = train.iloc[:, 1:]
# y = train.iloc[:, 0]

# #Loading test set
# test = pd.read_csv("mnist_test.csv", delimiter= ",")
# x_test = test.iloc[:, 1:]
# y_test = test.iloc[:, 0]

# # scaler = preprocessing.StandardScaler().fit(X)

# # X = scaler.transform(X)
# # test_X = scaler.transform( test_X )

# scaler = preprocessing.StandardScaler()
# X = scaler.fit_transform( X )
# x_test = scaler.transform(x_test)


# sizes2 = [784,30, 10]
# EPOCHS = 50

# net1 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = MSE(),
#             epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.05)
        
# net2 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
#             epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.05)





In [4]:
from sklearn import preprocessing
from sklearn import datasets

iris = datasets.load_iris()
X_iris = iris.data 
y_iris= iris.target


scaler = preprocessing.StandardScaler().fit(X_iris)

X_scaled = scaler.transform(X_iris)

sizes = [4, 10, 3]
EPOCHS = 50
net3 = Dense(sizes, activations = [Relu(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
                 epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.015)


In [5]:
net3.predict(X_scaled[:5], y[:5])

NameError: name 'y' is not defined

In [193]:
train = pd.read_csv("mnist_train.csv", delimiter= ",")
X = train.iloc[:, 1:]
y = train.iloc[:, 0]

#Loading test set
test = pd.read_csv("mnist_test.csv", delimiter= ",")
x_test = test.iloc[:, 1:]
y_test = test.iloc[:, 0]

# scaler = preprocessing.StandardScaler().fit(X)

# X = scaler.transform(X)
# test_X = scaler.transform( test_X )

scaler = preprocessing.StandardScaler()
X = scaler.fit_transform( X )
x_test = scaler.transform(x_test)


sizes2 = [784,30, 10]
EPOCHS = 50

net1 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = MSE(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.05)
        
net2 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.05)

In [24]:
#Testing predict module

import pandas as pd
path = r"C:\Users\malfaro\Desktop\mae_code\NeuralNetworksMae\datasets\diabetes\diabetes_total_vieja.csv"
df = pd.read_csv(path)

X = df.iloc[:, :-1]
y = df.iloc[:, -1]



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

#Scaling
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)


In [31]:
sizes2 = [8,80, 2]
EPOCHS = 500
minibatch_size = 10

net1 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = MSE(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.05)
        
net2 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.85)

In [34]:
net1.SGD(X_train,y_train,X_test, y_test, minibatch_size)
#net2.Momentum(X_train,y_train,X_test, y_test, minibatch_size, beta = 0.9)

SGD Training......
Average Error epoch 100: 0.10066244496334997---Average Accuracy: 0.8543478260869565
Accuracy in test set: 0.8311688311688312
Average Error epoch 200: 0.09872757848048995---Average Accuracy: 0.8586956521739131
Accuracy in test set: 0.8506493506493507
Average Error epoch 300: 0.09622582124207944---Average Accuracy: 0.8695652173913043
Accuracy in test set: 0.8506493506493507
Average Error epoch 400: 0.09399239450968676---Average Accuracy: 0.8804347826086955
Accuracy in test set: 0.8376623376623377
Average Error epoch 500: 0.09194629067836349---Average Accuracy: 0.8782608695652172
Accuracy in test set: 0.8441558441558441
Training complete!


In [35]:
net1.predict(X_val, y_val)

Predictive accuracy: 0.8181818181818182


In [12]:
#Mulanic
path = r"C:\Users\malfaro\Desktop\mae_code\NeuralNetworksMae\datasets\breastCancer\breastCancer.csv"

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

#Scaling
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [13]:
sizes2 = [8,30, 2]
EPOCHS = 25000
minibatch_size = 10

net1 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = MSE(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.05)
        
net2 = Dense(sizes2, activations = [Sigmoid(), Softmax()], Loss = CategoricalCrossEntropyLoss(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = 0.005)

In [14]:
net1.SGD(X_train,y_train,X_test, y_test, minibatch_size)
print("Validation...")
print("------------------------------------------------------------------")
net2.predict(X_val, y_val)

SGD Training......
Average Error epoch 100: 0.02579886782999313---Average Accuracy: 0.8195652173913043
Error booster: 0.07727482044862177
Accuracy in test set: 0.8116883116883117
Average Error epoch 200: 0.02451710191035509---Average Accuracy: 0.8413043478260869
Error booster: 0.06482130517257552
Accuracy in test set: 0.8376623376623377
Average Error epoch 300: 0.023937066560293235---Average Accuracy: 0.8347826086956521
Error booster: 0.06872694109670759
Accuracy in test set: 0.8441558441558441
Average Error epoch 400: 0.023400513498336984---Average Accuracy: 0.8391304347826086
Error booster: 0.0720953670215557
Accuracy in test set: 0.8246753246753247
Average Error epoch 500: 0.022813804836479677---Average Accuracy: 0.8369565217391303
Error booster: 0.07535143677094511
Accuracy in test set: 0.8311688311688312
Average Error epoch 600: 0.022232217780422642---Average Accuracy: 0.8391304347826086
Error booster: 0.07777282036882796
Accuracy in test set: 0.8116883116883117
Average Error epoc

In [13]:
cart_data = np.load("saved_training_data.npy",  allow_pickle=True)

In [14]:
def get_data(data):
    """Splits the generated dataset into X and y for training purposes"""
    X = np.array([data[i][0] for i in range(len(data))])
    y = np.array([data[i][1] for i in range(len(data))])
    return X, y

In [171]:
X, y = get_data(cart_data)

y_t = np.argmax(y)
y_t = []
for i in y:
    y_t.append(np.argmax(i))


X_train, X_test, y_train, y_test = train_test_split(X, np.array(y_t), test_size=0.3, random_state=1)

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

#Scaling
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# (unique, counts) = np.unique(y_t, return_counts=True)
# frequencies = np.asarray((unique, counts/len(y_t))).T
# frequencies


In [184]:
sizes2 = [4, 128, 256, 128, 2]
EPOCHS = 500
minibatch_size = 10

net1 = Dense(sizes2, activations = [Sigmoid(),Sigmoid(), Softmax()], Loss = MSE(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = 3)
        
net2 = Dense(sizes2, activations = [Sigmoid(),Sigmoid(),Sigmoid(),Softmax()], Loss = CategoricalCrossEntropyLoss(),
            epochs = EPOCHS, metric = Accuracy(), learning_rate = 2)

In [185]:
nr = 5000
net2.SGD(X_train[:nr],y_train[:nr],X_test[:nr], y_test[:nr], minibatch_size)

SGD Training......
Average Error epoch 100: 8.104178543021847---Average Accuracy: 0.49720000000000003
Accuracy in test set: 0.5034
Average Error epoch 200: 8.104178543021847---Average Accuracy: 0.49720000000000003
Accuracy in test set: 0.5034


KeyboardInterrupt: 

In [102]:
net1.predict(X_val, y_val)

Predictive accuracy: 0.46411067857843


In [152]:
net1.prediction[:5]

array([[0.57528022, 0.42471978],
       [0.57528022, 0.42471978],
       [0.36188863, 0.63811137],
       [0.57528022, 0.42471978],
       [0.36188863, 0.63811137]])

In [153]:
import gym
import random
# import numpy as np
# import tflearn
# from tflearn.layers.core import input_data, dropout, fully_connected
# from tflearn.layers.estimator import regression
# from statistics import median, mean
# from collections import Counter

LR = 1e-3
env = gym.make("CartPole-v0")
env.reset()


array([ 0.01361164, -0.00534999, -0.0108652 , -0.03312088], dtype=float32)

In [154]:
scores = []
choices = []
goal_steps = 200
score_requirement = 50
for each_game in range(5000):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            #action = np.argmax(net2.forward(prev_obs.reshape(-1,len(prev_obs),1))[0])
            action = np.argmax(net2.forward(prev_obs))

        choices.append(action)
                
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score+=reward
        if done: break

    scores.append(score)

print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
print(score_requirement)

Average Score: 10.4254
choice 1:0.04820918142229555  choice 0:0.9517908185777044
50
